In [ ]:
# Przetwarzanie języka naturalnego
# zadanie 2
# autor: Marcin Świątkowski

In [2]:
# punkt 1:
# !pip install elasticsearch==7.10.0

     |████████████████████████████████| 321 kB 949 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
# punkt 2:
# !pip install elasticsearch_dsl

     |████████████████████████████████| 63 kB 6.1 kB/s eta 0:00:01:55


In [12]:
import elasticsearch
import elasticsearch_dsl
import requests
import json
import os

In [2]:
# tworzymy połączenie
es = elasticsearch.Elasticsearch()
elasticsearch_dsl.connections.add_connection('python_client', es)
es.info()

{'name': 'marcin-Latitude-E7440',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'iftGRsZpRl2uOzIV_bv8yg',
 'version': {'number': '7.10.0',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': '51e9d6f22758d0374a0f3f5c6e8f3a7997850f96',
  'build_date': '2020-11-09T21:30:33.964949Z',
  'build_snapshot': False,
  'lucene_version': '8.7.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [3]:
print(es.cat.plugins(params={'v': '', 'h': 'name,component,version,description'}))

name                  component           version description
marcin-Latitude-E7440 analysis-morfologik 7.10.0  Morfologik Polish Lemmatizer plugin for Elasticsearch



In [5]:
# sprawdzamy czy indeks istnieje
INDEX_NAME = 'acts'
index_exists = es.indices.exists(index=INDEX_NAME)
index_exists

True

In [7]:
# punkt 3: definiujemy analyzer
analyzer = elasticsearch_dsl.analyzer(
    'bills_analyzer',
    type='custom',
    tokenizer='standard',                             
    filter=[
        'lowercase',                                  
        elasticsearch_dsl.analysis.token_filter(      
            'synonym_filter', 
            type='synonym', 
            synonyms=[
                'kpk,kodeks postępowania karnego',
                'kpc,kodeks postępowania cywilnego',
                'kk,kodeks karny',
                'kc,kodeks cywilny',
            ],
        ),
        'morfologik_stem'                            
    ]
)

In [8]:
# punkt 4: tworzymy klasę indeksu
class Act(elasticsearch_dsl.Document):
    content = elasticsearch_dsl.Text(
        analyzer=analyzer
    )

    class Index:
        name = 'acts'

In [9]:
# sprawdzamy czy indeks istnieje. Jeśli tak - usuwamy, a potem tworzymy
if Act._index.exists(using=es):
    Act._index.delete(using=es)

Act.init(using=es)

In [10]:
# funkcja do pozyskiwania tekstu z pliku
def getText(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        output = f.read()
        return output

In [13]:
path = "/home/marcin/Pulpit/Studia semestr 2/NLP/lab2/ustawy"
os.chdir(path)

# punkt 5: ładujemy dane do indeksu
for index, file in enumerate(os.listdir()):
    if file.endswith(".txt"):
        file_path = f"{path}/{file}"
        requests.post(f'http://localhost:9200/acts/_doc/{index}', json={"content" : getText(file_path)})
        print("Done request", index)

Done request 0
Done request 1
Done request 2
Done request 3
Done request 4
Done request 5
Done request 6
Done request 7
Done request 8
Done request 9
Done request 10
Done request 11
Done request 12
Done request 13
Done request 14
Done request 15
Done request 16
Done request 17
Done request 18
Done request 19
Done request 20
Done request 21
Done request 22
Done request 23
Done request 24
Done request 25
Done request 26
Done request 27
Done request 28
Done request 29
Done request 30
Done request 31
Done request 32
Done request 33
Done request 34
Done request 35
Done request 36
Done request 37
Done request 38
Done request 39
Done request 40
Done request 41
Done request 42
Done request 43
Done request 44
Done request 45
Done request 46
Done request 47
Done request 48
Done request 49
Done request 50
Done request 51
Done request 52
Done request 53
Done request 54
Done request 55
Done request 56
Done request 57
Done request 58
Done request 59
Done request 60
Done request 61
Done request 62
Do

Done request 489
Done request 490
Done request 491
Done request 492
Done request 493
Done request 494
Done request 495
Done request 496
Done request 497
Done request 498
Done request 499
Done request 500
Done request 501
Done request 502
Done request 503
Done request 504
Done request 505
Done request 506
Done request 507
Done request 508
Done request 509
Done request 510
Done request 511
Done request 512
Done request 513
Done request 514
Done request 515
Done request 516
Done request 517
Done request 518
Done request 519
Done request 520
Done request 521
Done request 522
Done request 523
Done request 524
Done request 525
Done request 526
Done request 527
Done request 528
Done request 529
Done request 530
Done request 531
Done request 532
Done request 533
Done request 534
Done request 535
Done request 536
Done request 537
Done request 538
Done request 539
Done request 540
Done request 541
Done request 542
Done request 543
Done request 544
Done request 545
Done request 546
Done request 5

Done request 971
Done request 972
Done request 973
Done request 974
Done request 975
Done request 976
Done request 977
Done request 978
Done request 979
Done request 980
Done request 981
Done request 982
Done request 983
Done request 984
Done request 985
Done request 986
Done request 987
Done request 988
Done request 989
Done request 990
Done request 991
Done request 992
Done request 993
Done request 994
Done request 995
Done request 996
Done request 997
Done request 998
Done request 999
Done request 1000
Done request 1001
Done request 1002
Done request 1003
Done request 1004
Done request 1005
Done request 1006
Done request 1007
Done request 1008
Done request 1009
Done request 1010
Done request 1011
Done request 1012
Done request 1013
Done request 1014
Done request 1015
Done request 1016
Done request 1017
Done request 1018
Done request 1019
Done request 1020
Done request 1021
Done request 1022
Done request 1023
Done request 1024
Done request 1025
Done request 1026
Done request 1027
Don

In [14]:
# punkt 6: znajdujemy liczbę dokumentów zawierających słowo 'ustawa' w dowolnej formie
contain_ustawa = es.search(
      index=Act.Index.name, 
      body={
        "query": {
            "match_phrase": {
                "content": {
                    "query": 'ustawa'
                }
            }
        }
      }
  )
contain_ustawa['hits']['total']['value']

1178

In [28]:
# punkt 7: znajdujemy liczbę wystąpień słowa 'ustawa' w dowolnej formie
search = requests.get('http://localhost:9200/acts/_termvectors/101', json={
    "fields": ["content"],
    "term_statistics": True
})
search.json()["term_vectors"]["content"]["terms"]["ustawa"]["ttf"]

24934

In [32]:
# punkt 8: znajdujemy liczbę wystąpień słowa 'ustaw' w dowolnej formie (działało za pomocą słowa 'ustawić')
search = requests.get('http://localhost:9200/acts/_termvectors/156', json={
    "fields": ["content"],
    "term_statistics": True
})
search.json()["term_vectors"]["content"]["terms"]["ustawić"]["ttf"]

913

In [39]:
# punkt 9: znajdujemy liczbę wystąpień słowa 'kodeks postępowania cywilnego' w dowolnej formie
search = requests.get('http://localhost:9200/acts/_termvectors/456', json={
    "fields": ["content"],
    "term_statistics": True
})
search.json()["term_vectors"]["content"]["terms"]["kodeks postępowania cywilnego"]["ttf"]

88

In [40]:
# punkt 10: znajdujemy liczbę aktów zawierających słowa 'wchodzi w życie' pozwalając na dwa współwystępujące słowa
search = es.search(
      index=Act.Index.name, 
      body={
        "query": {
            "match_phrase": {
                "content": {
                    "query": "wchodzi w życie",
                    "slop": 2
                }
            }
        }
      }
  )
search['hits']['total']['value']

1174

In [46]:
# punkt 11: znajdujemy dziesięć aktów najbardziej odnoszących się do słowa 'konstytucja'
contain_ustawa = requests.get(url="{}/acts/_search?filter_path=hits.hits._id,hits.hits._score".format('http://localhost:9200'), json={
        "query": {
            "match": {
                "content": {
                    "query": "konstytucja"
                    
                }
            }
        },
    "size": 10
      }
  )
contain_ustawa.json()['hits']['hits']

[{'_id': '583', '_score': 6.869376},
 {'_id': '257', '_score': 6.6642833},
 {'_id': '549', '_score': 6.633483},
 {'_id': '968', '_score': 6.628302},
 {'_id': '413', '_score': 6.2530584},
 {'_id': '634', '_score': 6.0589767},
 {'_id': '396', '_score': 5.9289904},
 {'_id': '155', '_score': 5.8507533},
 {'_id': '249', '_score': 5.467437},
 {'_id': '497', '_score': 5.467437}]

In [47]:
# punkt 12: wyświetlamy po trzy ustępy z aktów z punktu 11
search = requests.get(
    url="{}/acts/_search?filter_path=hits.hits._id,hits.hits.highlight".format('http://localhost:9200'),
    json={
        "query": {
            "match": {
                "content": {
                    "query": "konstytucja"
                }
            }
        },
        "highlight": {
            "fields": {
                "content": {
                    "number_of_fragments": 3
                }
            }
        },
        "size": 10
    }
)

search.json()['hits']['hits']

[{'_id': '583',
  'highlight': {'content': ['o zmianie ustawy konstytucyjnej o trybie przygotowania\n           i uchwalenia <em>Konstytucji</em> Rzeczypospolitej',
    'W ustawie  konstytucyjnej z  dnia 23 kwietnia 1992 r. o trybie przygotowania i \nuchwalenia <em>Konstytucji</em>',
    'Do zgłoszenia projektu <em>Konstytucji</em> załącza się wykaz \n                obywateli popierających zgłoszenie']}},
 {'_id': '257',
  'highlight': {'content': ['umowy międzynarodowej i nie wypełnia przesłanek określonych w art. 89\n     ust. 1 lub art. 90 <em>Konstytucji</em>',
    'międzynarodowej lub załącznika nie\n     wypełnia przesłanek określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em>',
    'co do zasadności wyboru\n  trybu ratyfikacji umowy międzynarodowej, o którym mowa w art. 89 ust. 2\n  <em>Konstytucji</em>']}},
 {'_id': '549',
  'highlight': {'content': ['Jeżeli Trybunał Konstytucyjny wyda orzeczenie o sprzeczności celów partii \n   politycznej z <em>Konstytucją</em>',
   